In [1]:
#!pip install librosa
#!pip install librosa==0.8.0

In [2]:
import librosa
import soundfile
import os,glob, pickle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft = abs(librosa.stft(X))
            result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr= sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S = stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result

In [4]:
emotions ={
    '01':'netural',
    '02':'clam',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'suprised',
} 

observed_emotions=['clam', 'happy', 'fearful', 'disgust']

In [5]:
def load_data(test_size=0.2):
    x,y = [],[]
    for file in glob.glob("C:\\Users\\aa22aon\\OneDrive - University of Hertfordshire\\projects\\Untitled Folder\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):
        file_name = os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [6]:
x_train, x_test, y_train, y_test= load_data(test_size=0.25)

In [7]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [8]:
print(f'Feature extracted : {x_train.shape[1]}')

Feature extracted : 180


In [9]:
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), 
                    learning_rate='adaptive', max_iter=500)

In [10]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [11]:
y_pred=model.predict(x_test)

In [12]:
y_pred

array(['happy', 'clam', 'happy', 'happy', 'fearful', 'clam', 'clam',
       'disgust', 'clam', 'happy', 'happy', 'clam', 'fearful', 'happy',
       'disgust', 'happy', 'clam', 'disgust', 'disgust', 'disgust',
       'clam', 'disgust', 'disgust', 'clam', 'happy', 'happy', 'happy',
       'happy', 'clam', 'happy', 'happy', 'disgust', 'happy', 'clam',
       'happy', 'clam', 'clam', 'fearful', 'clam', 'clam', 'happy',
       'clam', 'clam', 'clam', 'fearful', 'clam', 'disgust', 'clam',
       'clam', 'happy', 'fearful', 'fearful', 'clam', 'fearful', 'happy',
       'clam', 'disgust', 'happy', 'clam', 'clam', 'disgust', 'clam',
       'happy', 'clam', 'disgust', 'clam', 'clam', 'disgust', 'disgust',
       'happy', 'disgust', 'fearful', 'fearful', 'fearful', 'fearful',
       'disgust', 'fearful', 'happy', 'clam', 'fearful', 'disgust',
       'clam', 'disgust', 'clam', 'disgust', 'clam', 'disgust', 'clam',
       'happy', 'fearful', 'disgust', 'disgust', 'clam', 'happy',
       'disgust', 

In [13]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

In [14]:
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 73.96%


In [15]:
accuracy

0.7395833333333334